In [1]:
from scipy import sparse
from collections import Counter
import numpy as np

In [2]:
# lil_matrix 使用两个列表保存非零元素，data保存每行中的非零元素，rows保存非零元素所在的列。
# 这种格式很适合逐个添加元素，并且能快速获取行相关的数据

b = sparse.lil_matrix((10,5))
b[2,3] = 1.0
b[3,4] = 2.0
b[3,2] = 3.0

In [3]:
print(b.data)

[list([]) list([]) list([1.0]) list([3.0, 2.0]) list([]) list([]) list([])
 list([]) list([]) list([])]


In [4]:
print(b.rows)

[list([]) list([]) list([3]) list([2, 4]) list([]) list([]) list([])
 list([]) list([]) list([])]


In [5]:
# 根据提供的预料库，构建此表
def build_vocab(corpus):
    '''
    build a vocabulary with word frequencies for an entire corpus
    return a dic 'w -> (i,f)'
    '''
    print('building vocab from corpus')

    vocab = Counter()
    for line in corpus:
        tokens = line.split()
        vocab.update(tokens)

    print('Done building vocab from corpus')
    return {word:(i,freq) for i,(word,freq) in enumerate(vocab.items())}

In [6]:
# 构建共现矩阵
def build_cooccur(vocab,corpus,window_size=5):
    '''
    buil a word co-occurrence list for the given corpus
    :param vocab:
    :param corpus:
    :param window_size:
    :param min_count:
    :return:
    '''
    vocab_size = len(vocab)
    id2word = dict((i,word) for word,(i,_) in vocab.items())

    # collect cooccurrences internally as a sparse matrix for passable
    # indexing speed;we will convert into a list later
    cooccurrences = sparse.lil_matrix((vocab_size,vocab_size),dtype=np.float64)

    for i,line in enumerate(corpus):
        if i % 1000 == 0:
            print('building cooccurrence matrix: on line %i',i)

        tokens = line.strip().split()
        token_ids = [vocab[word][0] for word in tokens]

        for center_i,center_id in enumerate(token_ids):
            # collect all word ids in left window of center word
            context_ids = token_ids[max(0,center_i - window_size):center_i]
            contexts_len = len(context_ids)

            for left_i,left_id in enumerate(context_ids):
                # distance from center word
                distance = contexts_len - left_i

                # Weight by inverse of distance between words
                increment = 1.0/float(distance)

                # build co-occurrence matrix symmetrically
                cooccurrences[center_id,left_id] += increment
                cooccurrences[left_id,center_id] += increment

    return cooccurrences

In [7]:
x_max = 100
alpha = 0.75

def fw(X_c_s):
    return (X_c_s/x_max) ** alpha if X_c_s < x_max else 1

In [8]:
# coo_matrix.data 存储的是每行中的非零元素
# coo_matrix.rows 保存非零元素所在的列
from torch.utils.data import DataLoader,Dataset
import torch
class TrainData(Dataset):
    def __init__(self,coo_matrix,vocab,id2word,min_count=5):
        '''
        coo_matrix:sparse.lil_matrix
        vocab: word -> (index,freq)
        '''
        # 将coo_matrix的pair 以及权重存储到列表中
        # ((i,j),X_ij)
        self.coo_len = 0
        coo_matrix_list = []
        for i,(row,data) in enumerate(zip(coo_matrix.rows,coo_matrix.data)):
            # 第i个单词，需要确定它是否<= min_count
            word_i = id2word[i]
            if min_count is not None and vocab[word_i][1] < min_count:
                continue
            for (j,x) in zip(row,data):
                word_j = id2word[j]
                if min_count is not None and vocab[word_j][1] < min_count:
                    continue
                coo_matrix_list.append(((i,j),x))
        # 为了方便处理，将c,s,X_c_s,W_c_s都变成numpy矩阵
        c_list = []
        s_list = []
        X_c_s_list = []
        W_c_s_list = []
        for ((c,s),x) in coo_matrix_list:
            self.coo_len += 1
            c_list.append(c)
            s_list.append(s)
            X_c_s_list.append(x)
            W_c_s_list.append(fw(x))
        # 转换成numpy
        c_array = np.array(c_list)
        s_array = np.array(s_list)
        X_c_s_array = np.array(X_c_s_list)
        W_c_s_array = np.array(W_c_s_list)
        self.c = torch.from_numpy(c_array).long()
        self.s = torch.from_numpy(s_array).long()
        self.X_c_s = torch.from_numpy(X_c_s_array).double()
        self.W_c_s = torch.from_numpy(W_c_s_array).double()
    
    def __len__(self):
        return self.coo_len
    
    def __getitem__(self,index):
        c = self.c[index]
        s = self.s[index]
        X_c_s = self.X_c_s[index]
        W_c_s = self.W_c_s[index]
        return c,s,X_c_s,W_c_s

In [9]:
corpus_file_name = '../data/mini_content_process.txt'
corpus = []
with open(corpus_file_name, 'r') as f:
    for line in f:
        corpus.append(line.strip())

vocab = build_vocab(corpus)
id2word = dict((i,word) for word,(i,_) in vocab.items())
coo_matrix = build_cooccur(vocab,corpus,window_size=3)
train_data = TrainData(coo_matrix,vocab,id2word,min_count=5)
vocab_size = len(vocab)

building vocab from corpus
Done building vocab from corpus
building cooccurrence matrix: on line %i 0


In [10]:
vocab_size

7256

In [11]:
data_loader = DataLoader(train_data,batch_size=2,shuffle=True)

In [12]:
c = None
s = None
X_c_s = None
W_c_s = None
for i,(batch_c,batch_s,batch_X_c_s,batch_W_c_s) in enumerate(data_loader):
    c = torch.LongTensor(batch_c)
    s = torch.LongTensor(batch_s)
    X_c_s = torch.DoubleTensor(batch_X_c_s)
    W_c_s = torch.DoubleTensor(batch_W_c_s)
    break

In [13]:
c

tensor([3969,  753])

In [14]:
s

tensor([1032,  139])

In [15]:
X_c_s

tensor([3.0000, 0.3333], dtype=torch.float64)

In [16]:
W_c_s

tensor([0.0721, 0.0139], dtype=torch.float64)

In [17]:
from torch import nn
embedding = nn.Embedding(vocab_size,2)

In [18]:
embedding(c)

tensor([[-1.3956, -2.0539],
        [ 0.2383,  1.3230]], grad_fn=<EmbeddingBackward>)

In [ ]:
class Glove(nn.Module):
    def __init__(self,vocab_size,vector_size):
        super(Glove,self).__init__()
        # center words weight and bias
        self.center_weight = nn.Embedding(vocab_size,vector_size)
        self.center_biase = nn.Embedding(vocab_size,1)

        # context words weight and bias
        self.context_weight = nn.Embedding(vocab_size,vector_size)
        self.context_biase = nn.Embedding(vocab_size,1)

    def forward(self,center_ids,context_ids):
        '''
        cal v_i^Tv_k _ b_i + b_k
        :param center_ids: [batch]
        :param context_ids: [batch]
        :return:
        '''
        # [batch,vector_size]
        print('center_ids')
        print(type(center_ids))
        print(center_ids)
        center_w = self.center_weight[center_ids]
        # [batch,1]
        center_b = self.center_biase[center_ids]


        context_w = self.context_weight[center_ids]
        context_b = self.context_biase[center_ids]

        # [batch,1]
        return torch.sum(center_w.mul(context_w),1,keepdim=True) + center_b + context_b


In [19]:

center_weight = nn.Embedding(vocab_size,2)
center_biase = nn.Embedding(vocab_size,1)

context_weight = nn.Embedding(vocab_size,2)
context_biase = nn.Embedding(vocab_size,1)

In [22]:
center_w = center_weight(c)
center_b = center_biase(c)

context_w = context_weight(s)
context_b = context_biase(s)